In [2]:
# note: using Tensorflow version 1.15
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import datetime
import numpy as np

Using TensorFlow backend.


In [3]:
epochs = 10
batch_size = 50

In [20]:
# input image dimensions
img_rows, img_cols = 28, 28

# Import data
# Gives 60000 training images, 10000 test images
# https://keras.io/datasets/#mnist-database-of-handwritten-digits
(x_train, y_train), (x_test, y_test) = mnist.load_data()


print(y_train[0])

# Split train into 80% training, 20% validation
# https://medium.com/@mjbhobe/mnist-digits-classification-with-keras-ed6c2374bd0e
val_percent = 0.2
val_count = int(val_percent * x_train.shape[0])

(x_val, y_val) = (x_train[:val_count], y_train[:val_count])
(x_train, y_train) = (x_train[val_count:], y_train[val_count:])

# Reshape the data
x_train = x_train.reshape(48000,img_rows, img_cols,1)
x_val = x_val.reshape(val_count,img_rows, img_cols,1)
x_test = x_test.reshape(10000,img_rows, img_cols,1)

print(x_train[0].dtype)
print(x_train.shape, 'train samples')
print(x_val.shape[0], 'validation samples')
print(x_test.shape[0], 'test samples')

5
uint8
(48000, 28, 28, 1) train samples
12000 validation samples
10000 test samples


In [17]:
# one-hot encode the labels - we have 10 output classes (0,1,2,...,9)
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train[0:5])

[[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


# Problem 1

In [66]:
# Build the Convolutional Neural Network
# https://medium.com/@mjbhobe/mnist-digits-classification-with-keras-ed6c2374bd0e\

def build_model_1():
    model = Sequential()
    #Convolutions
    # use 32 5x5 filters
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=(img_rows,img_cols,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # use 64 5x5 filters
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    #Fully Connected Layer
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    #Output layer
    model.add(Dense(num_classes, activation='softmax'))

    #Adam optimizer, crossentropy as loss function
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam',metrics=['accuracy'])
    return model

# Kept getting error messages without this line
import keras.backend as K
K.clear_session()

model_1 = build_model_1()  

# Make Tensorboard logs
# https://www.tensorflow.org/tensorboard/get_started
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#Save best models
model_checkpoint = keras.callbacks.ModelCheckpoint('best_mnist_model.hdf5', monitor='val_loss', save_best_only=True, period=1)

results_1 = model_1.fit(x_train, y_train, 
                        batch_size=batch_size, 
                        epochs=epochs, 
                        verbose=1, 
                        validation_data=(x_val, y_val),
                        callbacks=[tensorboard_callback, model_checkpoint])


score = model_1.evaluate(x_test, y_test, verbose=0)


Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 23s 479us/step - loss: 1.5986 - acc: 0.8455 - val_loss: 0.0785 - val_acc: 0.9767
Epoch 2/10
48000/48000 [==============================] - 22s 465us/step - loss: 0.1073 - acc: 0.9698 - val_loss: 0.0813 - val_acc: 0.9781
Epoch 3/10
48000/48000 [==============================] - 24s 503us/step - loss: 0.0836 - acc: 0.9762 - val_loss: 0.0640 - val_acc: 0.9821
Epoch 4/10
48000/48000 [==============================] - 23s 478us/step - loss: 0.0703 - acc: 0.9800 - val_loss: 0.0542 - val_acc: 0.9850
Epoch 5/10
48000/48000 [==============================] - 22s 465us/step - loss: 0.0627 - acc: 0.9811 - val_loss: 0.0640 - val_acc: 0.9842
Epoch 6/10
48000/48000 [==============================] - 22s 454us/step - loss: 0.0596 - acc: 0.9840 - val_loss: 0.0724 - val_acc: 0.9831
Epoch 7/10
48000/48000 [==============================] - 23s 484us/step - loss: 0.0551 - acc: 0.9845 - val_loss: 0.

In [67]:
print('Training accuracy = ', results_1.history['acc'][epochs-1])
print('Validation accuracy = ', results_1.history['val_acc'][epochs-1])
print('Testing accuracy = ', score[1])

Training accuracy =  0.9871875047062834
Validation accuracy =  0.9863333384195964
Testing accuracy =  0.9879


# Problem 2

In [63]:
# Build the network
# https://medium.com/@mjbhobe/mnist-digits-classification-with-keras-ed6c2374bd0e
# https://medium.com/datadriveninvestor/image-processing-for-mnist-using-keras-f9a1021f6ef0

def build_model_2():
    model = Sequential()
    #Convolutions
    # use 32 5x5 filters
    model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same', input_shape=(img_rows,img_cols,1)))
    BatchNormalization(axis=-1)
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # use 64 5x5 filters
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    BatchNormalization(axis=-1)
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # use 32 3x3 filters
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    BatchNormalization(axis=-1)
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # use 32 3x3 filters
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    BatchNormalization(axis=-1)
    model.add(MaxPooling2D(pool_size=(2, 2)))

    #Fully connected layer
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    #Output layer
    model.add(Dense(num_classes, activation='softmax'))

    # Adam optimizer, crossentropy loss
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam',metrics=['accuracy'])
    return model


K.clear_session()

model_2 = build_model_2()


#Make tensorboard logs
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


#Save best models
model_checkpoint = keras.callbacks.ModelCheckpoint('best_mnist_model_2.hdf5', monitor='val_loss', save_best_only=True, period=1)


results_2 = model_2.fit(x_train, y_train, 
                        batch_size=batch_size, 
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_val, y_val), 
                        callbacks=[tensorboard_callback, model_checkpoint])


score = model_2.evaluate(x_test, y_test, verbose=0)


Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 60s 1ms/step - loss: 0.5310 - acc: 0.8400 - val_loss: 0.1050 - val_acc: 0.9695
Epoch 2/10
48000/48000 [==============================] - 60s 1ms/step - loss: 0.1073 - acc: 0.9717 - val_loss: 0.0895 - val_acc: 0.9742
Epoch 3/10
48000/48000 [==============================] - 60s 1ms/step - loss: 0.0848 - acc: 0.9774 - val_loss: 0.0736 - val_acc: 0.9806
Epoch 4/10
48000/48000 [==============================] - 60s 1ms/step - loss: 0.0687 - acc: 0.9815 - val_loss: 0.0730 - val_acc: 0.9792
Epoch 5/10
48000/48000 [==============================] - 60s 1ms/step - loss: 0.0646 - acc: 0.9827 - val_loss: 0.1098 - val_acc: 0.9699
Epoch 6/10
48000/48000 [==============================] - 59s 1ms/step - loss: 0.0581 - acc: 0.9840 - val_loss: 0.0673 - val_acc: 0.9810
Epoch 7/10
48000/48000 [==============================] - 60s 1ms/step - loss: 0.0528 - acc: 0.9855 - val_loss: 0.1023 - val_acc

In [64]:
print('Training accuracy = ', results_2.history['acc'][epochs-1])
print('Validation accuracy = ', results_2.history['val_acc'][epochs-1])
print('Testing accuracy = ', score[1])
# Note these are the accuracies in the final EPOCH

Training accuracy =  0.9877500037352244
Validation accuracy =  0.9832500040531158
Testing accuracy =  0.9882


In [68]:
# https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks
# https://theffork.com/how-to-use-tensorboard-in-jupyter-notebook/

# Run the following in command line (without #):
# pip install jupyter-tensorboard
# Load TENSORBOARD
%load_ext tensorboard
# Start TENSORBOARD
%tensorboard --logdir=./logs/

# Alternate method
# On a command line, run: tensorboard --logdir=ENTERLOGFOLDERPATH --host localhost --port 6066
# Then open http://localhost:6066 in your browser

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 1495), started 1:10:29 ago. (Use '!kill 1495' to kill it.)